In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline


df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')


In [3]:
df_train = df_train.drop(columns =['id'])
df_test= df_test.drop(columns= ['id'])

In [4]:
binary_features_names= ['Previously_Insured','Driving_License','Gender','Vehicle_Damage']
cont_features_names = ['Age','Annual_Premium','Vintage']
ordinal_features_names= ['Vehicle_Age',]

In [5]:
from sklearn import preprocessing
train_model = df_train.copy()
test_model = df_test.copy()



oe = preprocessing.OrdinalEncoder()
train_model[ordinal_features_names] = oe.fit_transform(train_model.filter(ordinal_features_names)).astype('int')
train_model[binary_features_names] = oe.fit_transform(train_model.filter(binary_features_names)).astype('int')

test_model[ordinal_features_names] = oe.fit_transform(test_model.filter(ordinal_features_names)).astype('int')
test_model[binary_features_names] = oe.fit_transform(test_model.filter(binary_features_names)).astype('int')


Region_code (-0.001, 26.0] < (26.0, 29.0] < (29.0, 52.0]
Policy_Sales_Channel [(0.999, 124.0], (124.0, 152.0], (152.0, 163.0]]


In [19]:
def Region_code_preprocessing(df):
    if df < 26:
        return 0
    elif 26<= df <29:
        return 1
    else:
        return 2
def policy_code_preprocsssing(df):
    if df < 124:
        return 0
    elif df < 152:
        return 1
    else:
        return 2

train_model['Region_Code'] = train_model['Region_Code'].apply(Region_code_preprocessing)
train_model['Policy_Sales_Channel'] = train_model['Policy_Sales_Channel'].apply(policy_code_preprocsssing)
test_model['Region_Code'] = test_model['Region_Code'].apply(Region_code_preprocessing)
test_model['Policy_Sales_Channel'] = test_model['Policy_Sales_Channel'].apply(policy_code_preprocsssing)


In [10]:
from sklearn.model_selection import train_test_split

X = train_model.loc[:, train_model.columns != "Response"]
y = train_model['Response']


X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.005, random_state=42)

print("Train set size:", X_train.shape)
print("Dev set size:", X_dev.shape)

Train set size: (11447274, 10)
Dev set size: (57524, 10)


In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# 최적의 하이퍼파라미터
best_params = {
    'alpha': 1.302348865795227e-06, 
    'max_depth': 15, 
    'learning_rate': 0.061800451723613786, 
    'subsample': 0.7098803046786328, 
    'colsample_bytree': 0.2590672912533101, 
    'min_child_weight': 10, 
    'gamma': 0.8399887056014855, 
    'reg_alpha': 0.0016943548302122801, 
    'max_bin': 71284,
}

# XGBoost 모델 초기화
best_xgb_model = XGBClassifier(
    **best_params, 
    n_estimators=12000, 
    random_state=42, 
    use_label_encoder=False,
    eval_metric="auc"
)

# 모델 학습
best_xgb_model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_dev, y_dev)], 
    verbose=500
)


/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [18:53:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.78888
[500]	validation_0-auc:0.88021
[1000]	validation_0-auc:0.88213
[1500]	validation_0-auc:0.88319
[2000]	validation_0-auc:0.88373
[2500]	validation_0-auc:0.88413
[3000]	validation_0-auc:0.88446
[3500]	validation_0-auc:0.88465
[4000]	validation_0-auc:0.88481
[4500]	validation_0-auc:0.88499
[5000]	validation_0-auc:0.88509
[5500]	validation_0-auc:0.88521
[6000]	validation_0-auc:0.88531
[6500]	validation_0-auc:0.88532
[7000]	validation_0-auc:0.88535
[7500]	validation_0-auc:0.88540
[8000]	validation_0-auc:0.88544
[8500]	validation_0-auc:0.88549
[9000]	validation_0-auc:0.88559
[9500]	validation_0-auc:0.88560
[10000]	validation_0-auc:0.88559
[10500]	validation_0-auc:0.88555
[11000]	validation_0-auc:0.88557
[11500]	validation_0-auc:0.88557
[11999]	validation_0-auc:0.88557


XGBClassifier(alpha=1.302348865795227e-06, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.2590672912533101, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0.8399887056014855,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.061800451723613786,
              max_bin=71284, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=12000, n_jobs=None,
              num_parallel_tree=None, ...)

In [17]:
y_pred_prob = best_xgb_model.predict_proba(X_dev)[:, 1]
score = roc_auc_score(y_dev, y_pred_prob)
print(score)

0.8855710323101429


In [20]:
preb_prob = best_xgb_model.predict_proba(test_model)[:, 1]

submission = pd.read_csv("/kaggle/input/playground-series-s4e7/sample_submission.csv")
submission["Response"] = preb_prob

submission.to_csv("submissiongboost_4.csv", index=False)
